# Steam's videogames platform 👾
## Company's description 📇

Steam is a video game digital distribution service and storefront from Valve. It was launched as a software client in September 2003 to provide game updates automatically for Valve's games, and expanded to distributing third-party titles in late 2005. Steam offers various features, like digital rights management (DRM), game server matchmaking with Valve Anti-Cheat measures, social networking, and game streaming services. Steam client's functions include game update automation, cloud storage for game progress, and community features such as direct messaging, in-game overlay functions and a virtual collectable marketplace.

## Project 🚧

You're working for Ubisoft, a French video game publisher. They'd like to release a new revolutionary videogame! They asked you conduct a global analysis of the games available on Steam's marketplace in order to better understand the videogames ecosystem and today's trends.

## Goals 🎯

The ultimate goal of this project is to understand what factors affect the popularity or sales of a video game. But your boss asked you to take advantage of this opportunity to analyze the video game market globally.

To carry out this project, you will have to adopt different levels of analysis. Your boss gave you a list of examples of questions that would be interesting:

### Analysis at the "macro" level

- Which publisher has released the most games on Steam?

- What are the best rated games?

- Are there years with more releases? Were there more or fewer game releases during the Covid, for example?

- How are the prizes distributed? Are there many games with a discount?

- What are the most represented languages?

- Are there many games prohibited for children under 16/18?

### Genres analysis

- What are the most represented genres?

- Are there any genres that have a better positive/negative review ratio?

- Do some publishers have favorite genres?

- What are the most lucrative genres?

### Platform analysis

- Are most games available on Windows/Mac/Linux instead?

- Do certain genres tend to be preferentially available on certain platforms?

You're free to follow these guidelines, or to choose a different angle of analysis, as long as your analysis reveals relevant and useful information. 🤓

## Scope of this project 🖼️

You'll have to use Databricks and PySpark to conduct this EDA. Particularly, you'll have to use Databrick's visualisation tool to create the visualizations.

The dataset is available in our S3 bucket at the following url: s3://full-stack-bigdata-datasets/Big_Data/Project_Steam/steam_game_output.json.

## Helpers 🦮

To help you achieve this project, here are a few tips that should help you:

- To adopt different levels of analysis, it might be useful to create different dataframes.

- As the dataset is semi-structured with a nested schema, Pyspark's methods such as getField() and explode() may help you.

- There are some text and date fields in this dataset: Pyspark offers utilitary functions to manipulate these types of data efficiently 💡

- You can use agregate functions and groupBy to conduct segmented analysis.

## Deliverable 📬

To complete this project, you should deliver:

- One or several notebooks including data manipulation with PySpark and data visualization with Databrick's dashboarding tool.

- To make sure the jury can view all the visualizations, please use the "publish" button on Databricks notebooks to create a public url where a copy of your notebook will be available.

- While using the "publish" button, Databricks may tell you that your notebook's size exceeds the maximal size allowed. If this happens, just split your notebook in several notebooks.

- Please copy-paste the link(s) to your published notebooks into your Github repo such that the jury can access it easily. 😌
  

In [0]:
sc = spark.sparkContext

from pyspark.sql import functions as F # This will load the class where spark sql functions are contained
from pyspark.sql import Row # this will let us manipulate rows with spark sql
from pyspark.sql.types import *

import pandas as pd

In [0]:
filepath = 's3://full-stack-bigdata-datasets/Big_Data/Project_Steam/steam_game_output.json'
steam = spark.read.json(filepath)

In [0]:
steam.printSchema()

root
 |-- data: struct (nullable = true)
 |    |-- appid: long (nullable = true)
 |    |-- categories: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- ccu: long (nullable = true)
 |    |-- developer: string (nullable = true)
 |    |-- discount: string (nullable = true)
 |    |-- genre: string (nullable = true)
 |    |-- header_image: string (nullable = true)
 |    |-- initialprice: string (nullable = true)
 |    |-- languages: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- negative: long (nullable = true)
 |    |-- owners: string (nullable = true)
 |    |-- platforms: struct (nullable = true)
 |    |    |-- linux: boolean (nullable = true)
 |    |    |-- mac: boolean (nullable = true)
 |    |    |-- windows: boolean (nullable = true)
 |    |-- positive: long (nullable = true)
 |    |-- price: string (nullable = true)
 |    |-- publisher: string (nullable = true)
 |    |-- release_date: string (nullable = true)
 |    |-

In [0]:
id_appid = steam.withColumn("are_equal", F.col("data.appid") == F.col("id"))

id_appid.select(F.col("are_equal")).distinct().show()

+---------+
|are_equal|
+---------+
|     true|
+---------+



The "id" column can be dropped since it is identical to the "appid" nested column in "data".

In [0]:
steam = steam.drop("id")
steam.printSchema()

root
 |-- data: struct (nullable = true)
 |    |-- appid: long (nullable = true)
 |    |-- categories: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- ccu: long (nullable = true)
 |    |-- developer: string (nullable = true)
 |    |-- discount: string (nullable = true)
 |    |-- genre: string (nullable = true)
 |    |-- header_image: string (nullable = true)
 |    |-- initialprice: string (nullable = true)
 |    |-- languages: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- negative: long (nullable = true)
 |    |-- owners: string (nullable = true)
 |    |-- platforms: struct (nullable = true)
 |    |    |-- linux: boolean (nullable = true)
 |    |    |-- mac: boolean (nullable = true)
 |    |    |-- windows: boolean (nullable = true)
 |    |-- positive: long (nullable = true)
 |    |-- price: string (nullable = true)
 |    |-- publisher: string (nullable = true)
 |    |-- release_date: string (nullable = true)
 |    |-

Now we can start unnesting the schema and flattening the dataset.

In [0]:
df = steam.select("*", "data.*").drop("data")

df.printSchema()

root
 |-- appid: long (nullable = true)
 |-- categories: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ccu: long (nullable = true)
 |-- developer: string (nullable = true)
 |-- discount: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- header_image: string (nullable = true)
 |-- initialprice: string (nullable = true)
 |-- languages: string (nullable = true)
 |-- name: string (nullable = true)
 |-- negative: long (nullable = true)
 |-- owners: string (nullable = true)
 |-- platforms: struct (nullable = true)
 |    |-- linux: boolean (nullable = true)
 |    |-- mac: boolean (nullable = true)
 |    |-- windows: boolean (nullable = true)
 |-- positive: long (nullable = true)
 |-- price: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- required_age: string (nullable = true)
 |-- short_description: string (nullable = true)
 |-- tags: struct (nullable = true)
 |    |-- 1980s: lon

In [0]:
df.limit(5).display()

appid,categories,ccu,developer,discount,genre,header_image,initialprice,languages,name,negative,owners,platforms,positive,price,publisher,release_date,required_age,short_description,tags,type,website
10,"List(Multi-player, Valve Anti-Cheat enabled, Online PvP, Shared/Split Screen PvP, PvP)",13990,Valve,0,Action,https://cdn.akamai.steamstatic.com/steam/apps/10/header.jpg?t=1666823513,999,"English, French, German, Italian, Spanish - Spain, Simplified Chinese, Traditional Chinese, Korean",Counter-Strike,5199,"10,000,000 .. 20,000,000","List(true, true, true)",201215,999,Valve,2000/11/1,0,Play the world's number 1 online action game. Engage in an incredibly realistic brand of terrorist warfare in this wildly popular team-based game. Ally with teammates to complete strategic missions. Take out enemy sites. Rescue hostages. Your role affects your team's success. Your team's success affects your role.,"List(266, 1191, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 5426, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 227, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 2784, null, null, null, null, null, null, null, null, null, null, null, null, 1607, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 4831, null, null, null, null, null, null, null, null, null, 1707, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 632, null, null, null, null, null, null, null, null, null, null, null, 3392, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 131, null, null, 769, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 881, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 289, null, null, null, 3353, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 614, null, null, null, null, null, null, 304, null, null, null, 1344, null, null, 1864, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 1192)",game,
1000000,"List(Single-player, Partial Controller Support, Steam Achievements, Steam Cloud)",0,IndigoBlue Game Studio,0,"Action, Adventure, Indie",https://cdn.akamai.steamstatic.com/steam/apps/1000000/header.jpg?t=1655723048,999,"English, Korean, Simplified Chinese",ASCENXION,5,"0 .. 20,000","List(false, false, true)",27,999,PsychoFlux Entertainment,2021/05/14,0,"ASCENXION is a 2D shoot 'em up game where you explore the field to progress. P

## 1. Macro-level analysis

### 1.1. Which publisher has released the most games on Steam?

In [0]:
df.groupby('publisher').count().sort(F.col('count').desc()).limit(1).show()

+--------------+-----+
|     publisher|count|
+--------------+-----+
|Big Fish Games|  422|
+--------------+-----+



"Big Fish Games" is the publisher who released the most games on Steam, with 442 available games on the platform.

In [0]:
top_publishers = df \
    .filter(df.publisher != "") \
    .groupBy("publisher") \
    .count() \
    .orderBy(F.col("count").desc()) \
    .limit(10)

display(top_publishers)

publisher,count
Big Fish Games,422
8floor,202
SEGA,165
Strategy First,151
Square Enix,141
Choice of Games,140
Sekai Project,132
HH-Games,132
Ubisoft,127
Laush Studio,126


Databricks visualization. Run in Databricks to view.

### 1.2. What are the best rated games?

On Steam, games ratings are calculated using the approval ratio of users.

In our dataset, each row represents a game, the number of positive and negative reviews are found respectively in the "positive" and "negative" columns.

Therefore, we'll create a "total" column with the total review count, then a "rating" column giving us the positive ratio.

In [0]:
top_rated = df \
    .withColumn("total", F.col("positive") + F.col("negative")) \
    .withColumn("rating", F.round(F.col("positive") / F.col("total"), 3))

In [0]:
top_rated \
    .select(["total", "rating"]) \
    .summary().show()


+-------+------------------+-------------------+
|summary|             total|             rating|
+-------+------------------+-------------------+
|  count|             55691|              55528|
|   mean|1712.7132929916863| 0.7365415285981882|
| stddev| 35687.61118456136|0.24231838102358305|
|    min|                 0|                0.0|
|    25%|                 7|              0.609|
|    50%|                26|              0.799|
|    75%|               145|              0.925|
|    max|           6730438|                1.0|
+-------+------------------+-------------------+



75% of the dataset's games have less than 145 reviews; meanwhile the game which gathered the most reviews reaches a total of 6,730,438!

That is a huge disparity that we should consider when looking at the best rated games : if game A has a 90% approval rating with 10,000 votes and game B has a 100% approval rating but only 100 votes, which game is better? Can a great ratio based on very few reviews be trusted? 

In [0]:
top_rated \
    .select(["name", "total", "rating"]) \
    .orderBy("rating", ascending=False) \
    .limit(20) \
    .display()

name,total,rating
The Spell - A Kinetic Novel,23,1.0
Twisting Mower,1,1.0
Diamonds,4,1.0
Cube Defender,6,1.0
Next Stop 3,17,1.0
Rotund Rebound,5,1.0
N-GON,4,1.0
Zero spring episode 3,1,1.0
Hand of Horzasha,10,1.0
Kooring VR Wonderland:Mecadino's Attack,12,1.0


As expected, the games with the best ratings all have 25 reviews or less. We'll therefore set a threshold of 80,000 reviews to rank best rated games.

In [0]:
 top_rated \
    .filter(F.col("total") > 80000) \
    .select(["name", "total", "rating"]) \
    .orderBy("rating", ascending=False) \
    .limit(20) \
    .display()

name,total,rating
People Playground,144569,0.989
Vampire Survivors,131935,0.988
Portal 2,309441,0.988
Hades,202789,0.986
Portal,113538,0.985
RimWorld,144751,0.982
Stardew Valley,506841,0.982
Wallpaper Engine,572127,0.981
Helltaker,108165,0.98
Slime Rancher,99306,0.979


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

### 1.3. Are there years with more releases? Were there more or fewer game releases during the Covid, for example?

In [0]:
release_date = df.select(F.col("release_date"),F.to_date(F.col("release_date"),"yyyy/M/d").alias("date"))
release_date = release_date.withColumn("year", F.year("date"))

release_year = release_date.groupby("year").count()

display(release_year)

year,count
2003,3
2007,98
2018,7663
2015,2566
2006,61
2022,7451
2013,469
null,222
1997,2
2014,1550


Databricks visualization. Run in Databricks to view.

Until 2003, Steam served only as a platform for Valve games, providing updates and multiplayer support. As such, only a handful of games were published of games in its early years.

In 2005, third-party developers were contracted to release games on Steam, thus growing the number of games offered on the platform.

Publishers such as id Software, Eidos Interactive, and Capcom began distributing their games on Steam in 2007, followed by the likes of Ubisoft, THQ, Sega, Take-Two Interactive, Activision, and Electronic Arts in 2008, thus beginning the exponential growth of the service as a digital distribution service.

Since then, only 2019 saw a slowdown in new releases. Many factors could be contributing to this, but one immediately comes to mind : Epic Games Store, a direct competitor, made its debut on December, 6th 2018. Valve, which operates Steam, takes a 30% revenue cut of all games sold through their services, a figure matched by the other services like GOG.com, and console and mobile storefronts. Epic Games opened their digital storefront to challenge Steam by using a 12% revenue split rather than Steam's 30%. This enticed developers and publishers to release their games on Epic Games Store rather than Steam, causing a dip in the amount of games published by Steam the following year.

It doesn't seem Covid (starting in 2020) had a particular effect in the number of releases. One explanation might be that many digital jobs like game developers and publishers are easily performed remotely, and Covid didn't cause too much trouble in the video game market.

### 1.4. How are the prices distributed? Are there many games with a discount?

In [0]:
df.select(["initialprice", "discount", "price"]).summary().show()

+-------+-----------------+------------------+-----------------+
|summary|     initialprice|          discount|            price|
+-------+-----------------+------------------+-----------------+
|  count|            55691|             55691|            55691|
|   mean|797.5663033524268| 2.603777989262179|773.2849832109317|
| stddev|1104.762477841338|12.887080174743176| 1093.13458272345|
|    min|                0|                 0|                0|
|    25%|            199.0|               0.0|            129.0|
|    50%|            499.0|               0.0|            499.0|
|    75%|            999.0|               0.0|            999.0|
|    max|             9999|                90|             9999|
+-------+-----------------+------------------+-----------------+



We easily notice that : 
- "initialprice" and "price" columns must be formatted : the values found are ludicrous for video game prices...unless they are divided by 100 ! For instance, a game where "price" value is 999.0 actually costs 9.99€.
- "discount" is a percentage
- In terms of distribution : 75% games on the platform cost less than 9.99€; less than 25% games have an active discount. 

In [0]:
df = df.withColumns({'initialprice': F.round(F.col("initialprice")/100, 2), 'price': F.round(F.col("price")/100, 2)})

prices = df.select(["initialprice", "discount", "price"])
prices_dist = prices \
    .select(["price"]) \
    .groupby('price') \
    .count() \
    .display()

price,count
4.19,11
0.84,3
44.99,44
19.96,1
9.49,1
8.0,3
0.0,7780
4.88,1
1.42,1
12.79,1


Since outliers prevent proper visualization, we'll remove them to get a better look at our prices distribution.

In [0]:
prices_dist = prices \
    .where(prices.price<60) \
    .select(["price"]) \

display(prices_dist)

price
9.99
9.99
5.99
19.99
1.99
7.99
12.99
0.0
2.99
13.99


Databricks visualization. Run in Databricks to view.

In [0]:
discount = df \
    .withColumn("is_discounted", F.when(df.discount == 0, "No discount").otherwise("Discounted")) \
    .groupby("is_discounted") \
    .count()
 
display(discount)

is_discounted,count
Discounted,2518
No discount,53173


Databricks visualization. Run in Databricks to view.

### 1.5. What are the most represented languages?

In [0]:
languages = df \
    .withColumn('language', F.explode(F.split(F.col("languages"), ", "))) \
    .groupBy('language') \
    .count() \
    .orderBy(F.col('count').desc())

languages.limit(3).display()

language,count
English,55116
German,14019
French,13426


Databricks visualization. Run in Databricks to view.

Unsurprisingly, English is by far the most represented language. Following are German in second and French in third, with close counts. 

### 1.6. Are there many games prohibited for children under 16/18?

In [0]:
age_restricted_16 = df \
    .filter(df["required_age"] >= 16) \
    .count()
age_restricted_18 = df \
    .filter(df["required_age"] >= 18) \
    .count()
print(f"There are {age_restricted_16} games in the dataset that are prohibited for children under 16, and {age_restricted_18} games forbidden to people under 18.")

There are 305 games in the dataset that are prohibited for children under 16, and 229 games forbidden to people under 18.


**PEGI-16** : 305 games

**PEGI-18** : 229 games

## 2. Genres analysis

###  2.1. What are the most represented genres?

In [0]:
df = df.withColumn("genre", F.explode(F.split(F.col("genre"), ", ")))

genres = df \
    .filter(df.genre != "") \
    .groupBy("genre") \
    .count() \
    .orderBy(F.col("count") \
    .desc()) \
    .limit(15)

display(genres)

genre,count
Indie,39681
Action,23759
Casual,22086
Adventure,21431
Strategy,10895
Simulation,10836
RPG,9534
Early Access,6145
Free to Play,3393
Sports,2666


Databricks visualization. Run in Databricks to view.

###  2.2. Are there any genres that have a better positive/negative review ratio?

In [0]:
genres_ratio = df \
    .filter(df.genre != "") \
    .select(["genre", "positive", "negative"]) \
    .groupby("genre") \
    .sum("positive", "negative") \
    .withColumn("ratio", F.round(F.col("sum(positive)") / F.col("sum(negative)"), 2)) \
    .select(F.col("genre"), F.col("ratio")) \
    .orderBy(F.col("ratio") \
    .desc()) \
    .limit(20)

display(genres_ratio)

genre,ratio
Photo Editing,42.03
Animation & Modeling,26.17
Design & Illustration,24.96
Utilities,17.0
Game Development,8.39
Indie,7.67
Audio Production,7.33
Video Production,6.82
Casual,6.53
Web Publishing,6.49


Databricks visualization. Run in Databricks to view.

###  2.3. Do some publishers have favorite genres?

In [0]:
pub_count = df \
    .filter(df.publisher != "") \
    .groupBy('publisher') \
    .agg(F.count(F.col('publisher')).alias('total_count'))
    
pub_genres = df \
    .filter(df.genre != "") \
    .join(pub_count, df['publisher'] == pub_count['publisher'],'inner') \
    .select(df['*'], pub_count['total_count']) \
    .groupBy(['publisher','genre','total_count']) \
    .agg(F.count(F.col('genre')).alias('genre_count')) \
    .withColumn('proportion', F.round(F.col('genre_count')/F.col('total_count'), 2)) \
    .orderBy(['proportion'], ascending = False)

display(pub_genres.filter(pub_genres.total_count > 10))

publisher,genre,total_count,genre_count,proportion
id Software,Action,16,16,1.0
"PopCap Games, Inc.",Casual,19,19,1.0
HexWar Games,Strategy,43,42,0.98
CITY CONNECTION,Action,19,18,0.95
grin robot,Indie,11,10,0.91
HeR Interactive,Adventure,33,29,0.88
Notus Games Ltd,Casual,13,11,0.85
8floor,Casual,243,202,0.83
Sandlot Games,Casual,11,9,0.82
SNK CORPORATION,Action,30,24,0.8


Databricks visualization. Run in Databricks to view.

###  2.4. What are the most lucrative genres?

In [0]:
sales = df \
    .filter(df.genre != "") \
    .groupBy('genre') \
    .agg(F.round(F.sum("price")).alias("total_sales")) \
    .orderBy(F.col('total_sales').desc()) \
    .limit(10)

display(sales)

genre,total_sales
Indie,260630.0
Action,183588.0
Adventure,171582.0
Casual,123836.0
Simulation,98517.0
Strategy,91572.0
RPG,86213.0
Early Access,53758.0
Sports,23855.0
Racing,17716.0


Databricks visualization. Run in Databricks to view.

In [0]:
price_per_game = df \
    .filter(df.genre != "") \
    .groupBy("genre") \
    .agg(F.round(F.mean("price"), 2) \
    .alias("avg_price_per_game")) \
    .orderBy(F.desc("avg_price_per_game")) \

display(price_per_game)

genre,avg_price_per_game
Web Publishing,21.8
Game Development,21.28
Photo Editing,20.33
Audio Production,19.64
Design & Illustration,19.06
Software Training,19.03
Video Production,18.96
Animation & Modeling,18.78
Accounting,14.43
Education,14.34


Databricks visualization. Run in Databricks to view.

It's worth noting that while indie may be the most profitable genre, indie games individual prices tend to be on the lower side . The indie genre represents many sales, simply because it is by far the most represented genre on the platform.

## 3. Platform analysis

###  3.1. Are most games available on Windows/Mac/Linux instead?

In [0]:
df = df \
    .withColumn("Linux", F.col("platforms")["linux"].cast('int')) \
    .withColumn( "MacOS", F.col("platforms")["mac"].cast('int')) \
    .withColumn("Windows", F.col("platforms")["windows"].cast('int'))

counts = df \
    .agg(F.sum(F.col("Linux")).alias("Linux"),
         F.sum(F.col('MacOS')).alias("MacOS"),
         F.sum(F.col('Windows')).alias("Windows"))
    
display(counts)

Linux,MacOS,Windows
23989,35902,157076


Databricks visualization. Run in Databricks to view.

In [0]:
pd_counts = counts.toPandas().transpose().reset_index().rename(columns={"index":'platform', 0:"available_games"})
t_counts = spark.createDataFrame(pd_counts)
tot = df.count()
platform_perc = t_counts \
    .withColumn("proportion_available", F.round(F.col("available_games")/tot * 100, 2))
display(platform_perc)

platform,available_games,proportion_available
Linux,23989,15.27
MacOS,35902,22.85
Windows,157076,99.98


Databricks visualization. Run in Databricks to view.

###  3.2. Do certain genres tend to be preferentially available on certain platforms?

In [0]:
genre_platforms = df \
    .groupBy("genre") \
    .agg(F.sum(F.col("Linux")).alias("Linux"),
         F.sum(F.col("MacOS")).alias("Mac"),
         F.sum(F.col("Windows")).alias("Windows"))

display(genre_platforms)

genre,Linux,Mac,Windows
Education,19,56,317
Massively Multiplayer,164,270,1459
Sexual Content,7,13,54
Adventure,3302,5039,21427
Sports,287,506,2665
Accounting,0,4,16
Audio Production,7,41,193
Video Production,6,29,247
Animation & Modeling,38,74,322
Racing,304,424,2154


Databricks visualization. Run in Databricks to view.

## Conclusion

Performing this analysis has given us a detailed view of Steam's ecosystem and a better understanding of the platform. 

It is crucial for publishers to finely tune their products to a demanding consumer base's expectations. Given Steam's domineering position in the computer games market, these findings can help publishers make strategic decisions such as : 
- when to release a game 
- what genres are currently in demand
- how to price their products
- who their direct competitors are
- whether porting to several platforms would be beneficial

and more!